In [1]:
import os
#os.environ["TF_GPU_THREAD_MODE"] = "gpu_private"

from pprint import pprint
import numpy as np
import tensorflow as tf
import keras
from keras.callbacks import Callback
from tensorflow.python.client import device_lib
from keras.applications import MobileNetV2
from keras.layers import Dense
from keras.models import Model
from keras.optimizers import Adam
# from keras.optimizers import SGD
from keras.utils import Sequence

# os.environ["TF_DATA_EXPERIMENTAL_OPTIMIZATION_AUTOTUNE_RAM_BUDGET"] = "18884901888"


gpu_options = tf.compat.v1.GPUOptions(per_process_gpu_memory_fraction=0.95)
sess = tf.compat.v1.Session(config=tf.compat.v1.ConfigProto(gpu_options=gpu_options))

# gpus = tf.config.experimental.list_physical_devices('GPU')
# if gpus:
#     try:
#         for gpu in gpus:
#             tf.config.experimental.set_memory_growth(gpu, True)
#         print("Memory growth enabled for GPUs")
#     except RuntimeError as e:
#         print(f"Error enabling memory growth: {e}")

print("TensorFlow version:", tf.__version__)
print("GPU Available:", tf.test.is_gpu_available())

# Get detailed device information
print(device_lib.list_local_devices())

2025-01-20 01:31:19.378713: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-01-20 01:31:19.410925: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-01-20 01:31:19.410964: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-01-20 01:31:19.411971: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-20 01:31:19.417309: I tensorflow/core/platform/cpu_feature_guar

TensorFlow version: 2.15.0
Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
GPU Available: True
[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 4587412722181471871
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 3784300953
locality {
  bus_id: 1
  links {
  }
}
incarnation: 11075676784906228803
physical_device_desc: "device: 0, name: NVIDIA GeForce RTX 3050 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.6"
xla_global_id: 416903419
]


2025-01-20 01:31:23.828894: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-01-20 01:31:23.876539: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-01-20 01:31:23.877610: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [2]:
# memmap_path='/mnt/Velocity_Vault/Autofocus/Dataset_Storage/'
# train_size=355
# patch_num=12
# test_size=47
# num_classes=49

# train_dataset = np.memmap(memmap_path+"train_dataset", dtype=np.float64, mode='r', shape=(patch_num*train_size,128, 128,98))
# train_truth = np.memmap(memmap_path+"train_truth_one_hot", dtype=int, mode='r', shape=(patch_num*train_size,num_classes))


In [3]:
memmap_path='/mnt/Velocity_Vault/Autofocus/Dataset_Storage/'
train_size=355
patch_num=12
test_size=47
num_classes=49

set_size_org=train_size*patch_num
set_size=3920

train_dataset = np.memmap(memmap_path+"resampled_dataset", dtype=np.float64, mode='r', shape=(set_size,128, 128,98))
train_truth = np.memmap(memmap_path+"resampled_truth_one_hot", dtype=int, mode='r', shape=(set_size,num_classes))


In [4]:
# memmap_path='/mnt/Velocity_Vault/Autofocus/Dataset_Storage/'
# train_size=355
# patch_num=12
# ratio=5
# test_size=47
# num_classes=49

# org_train_dataset = np.memmap(memmap_path+"train_dataset", dtype=np.float64, mode='r', shape=(patch_num*train_size,128, 128,98))
# org_train_truth = np.memmap(memmap_path+"train_truth_one_hot", dtype=int, mode='r', shape=(patch_num*train_size,num_classes))

# decreased_size=(train_size)//ratio

# train_dataset = np.memmap(memmap_path+"small_train_dataset", dtype=np.float64, mode='w+', shape=(patch_num*decreased_size,128, 128,98))
# train_truth = np.memmap(memmap_path+"small_train_truth_one_hot", dtype=int, mode='w+', shape=(patch_num*decreased_size,num_classes))

# train_dataset=org_train_dataset[:patch_num*decreased_size]
# train_truth=org_train_truth[:patch_num*decreased_size]

# train_dataset.flush()
# train_truth.flush()

# train_size=decreased_size





In [5]:
print(train_dataset.shape)
print(train_truth.shape)

(3920, 128, 128, 98)
(3920, 49)


In [6]:
memmap_path='/mnt/Velocity_Vault/Autofocus/Dataset_Storage/'
test_size=47
num_classes=49

test_dataset = np.memmap(memmap_path+"test_dataset", dtype=np.float64, mode='r', shape=(patch_num*test_size,128, 128,98))
test_truth = np.memmap(memmap_path+"test_truth", dtype=int, mode='r', shape=(patch_num*test_size))

In [7]:
def create_model(input_shape=(128, 128, 98), num_classes=49):
    base_model = MobileNetV2(input_shape=input_shape, include_top=False, pooling='avg', weights=None)
    x = Dense(256, activation='relu')(base_model.output)
    outputs = Dense(num_classes, activation='sigmoid')(x)
    model = Model(inputs=base_model.input, outputs=outputs)
    return model


def ordinal_loss(y_true, y_pred):
    y_true = tf.cast(y_true, tf.float32)
    return tf.reduce_mean(tf.square(y_true - y_pred),axis=1)


model = create_model()

model.compile(
    optimizer=Adam(learning_rate=6.25e-6, beta_1=0.5, beta_2=0.999),
    loss=ordinal_loss
)


# model.compile(
#     optimizer=SGD(learning_rate=0.01, momentum=0.9),
#     loss=ordinal_loss
# )


/mnt/Installation/Virtual_Environment/Python3_11/lib/python3.11/site-packages/keras/src/applications/mobilenet_v2.py:299: UserWarning: This model usually expects 1 or 3 input channels. However, it was passed an input_shape with 98 input channels.
  input_shape = imagenet_utils.obtain_input_shape(
2025-01-20 01:31:24.044019: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-01-20 01:31:24.045891: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-01-20 01:31:24.047577: I ex

In [8]:
# model_config = model.get_config()
# print(model_config)
# model.summary()

In [9]:
# set_size=set_size_org
batch_size = 8
global_steps = 320000
#512000
steps_per_epoch = set_size // batch_size  
epochs = global_steps // steps_per_epoch

print(set_size)
print(batch_size)
print(steps_per_epoch)
print(epochs)

3920
8
490
653


In [10]:
# class DataGenerator(Sequence):
#     def __init__(self, x_set, y_set, batch_size):
#         self.x, self.y = x_set, y_set
#         self.batch_size = batch_size

#     def __len__(self):
#         return int(np.ceil(len(self.x) / float(self.batch_size)))

#     def __getitem__(self, idx):
#         batch_x = self.x[idx * self.batch_size:(idx + 1) * self.batch_size]
#         batch_y = self.y[idx * self.batch_size:(idx + 1) * self.batch_size]
#         return batch_x, batch_y

# train_gen = DataGenerator(train_dataset, train_truth, batch_size)

In [ ]:
import numpy as np   

model_path='/mnt/Velocity_Vault/Autofocus/Model/Model_V1/model'

class CustomSaveCallback(Callback):
    def __init__(self, save_epochs, save_path):
        super().__init__()
        self.save_epochs = save_epochs
        self.save_path = save_path

    def on_epoch_end(self, epoch, logs=None):
        if (epoch + 1) in self.save_epochs:  # Check if the current epoch is in the list
            self.model.save(f"{self.save_path}_epoch_{epoch + 1}.keras")

# Specify the epochs where you want to save the model
save_epochs = [(i+1)*50 for i in range(13)]

custom_callback = CustomSaveCallback(save_epochs, model_path)


class DataGenerator(Sequence):
    def __init__(self, dataset, truth, batch_size=5):
        self.dataset = dataset
        self.truth = truth
        self.batch_size = batch_size
        self.indices = np.arange(len(self.dataset))

    def __len__(self):
        return int(np.ceil(len(self.dataset) / self.batch_size))

    def __getitem__(self, idx):
        batch_indices = self.indices[idx * self.batch_size : (idx + 1) * self.batch_size]
        batch_x = self.dataset[batch_indices]
        batch_y = self.truth[batch_indices]
        return batch_x, batch_y

    def on_epoch_end(self):
        # np.random.shuffle(self.indices)
        pass


train_gen = DataGenerator(train_dataset, train_truth, batch_size=batch_size)

hist=model.fit(train_gen, epochs=epochs,verbose=1,callbacks=[custom_callback])

# hist=model.fit(train_dataset[:100], train_truth[:100],batch_size=3,epochs=epochs)

model_path='/mnt/Velocity_Vault/Autofocus/Model/Model_V1.keras'
model.save(model_path)  # Save in SavedModel format




Epoch 1/653


2025-01-20 01:31:32.864755: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8904
2025-01-20 01:31:32.982176: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2025-01-20 01:31:35.341817: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2025-01-20 01:31:35.563385: I external/local_xla/xla/service/service.cc:168] XLA service 0x78f15ca3aa30 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2025-01-20 01:31:35.563407: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 3050 Laptop GPU, Compute Capability 8.6
2025-01-20 01:31:35.583589: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1737316895.699670   44663 devic

  9/490 [..............................] - ETA: 2:34 - loss: 0.2541

In [12]:
# def ordinal_loss(y_true, y_pred):
#     y_true = tf.cast(y_true, tf.float32)
#     return tf.reduce_mean(tf.square(y_true - y_pred),axis=1)

# from tensorflow.keras.models import load_model
# model_path='/mnt/Velocity_Vault/Autofocus/Model/Model_V1.keras'
# model=load_model(model_path,custom_objects={'ordinal_loss': ordinal_loss})

In [ ]:
#print(test_dataset)
print(test_dataset.shape)

In [ ]:

predictions=np.zeros((47 * 12, 49), dtype=np.float64)


for i in range(12):
    test_temp=test_dataset[i*47:(i+1)*47]
    pred_temp=model.predict(test_temp)
    for j in range(47):
        predictions[i*47+j]=pred_temp[j]

print(predictions.shape)
# pprint(predictions)




#predictions=model.predict(test_dataset)


In [15]:
def get_max_positions(array):
    # Find the index of the max value along each row
    return np.argmax(array, axis=1)

pred_truth=get_max_positions(predictions)

In [ ]:
def calculate_accuracy(array1, array2):
    # Check if both arrays are the same shape
    if array1.shape != array2.shape:
        raise ValueError("Arrays must have the same shape")
    
    # Compare the arrays and calculate the accuracy
    correct_predictions = np.sum(array1 == array2)
    total_elements = array1.size
    
    accuracy = correct_predictions / total_elements
    return accuracy*100

pred_test=calculate_accuracy(pred_truth,test_truth)

print("Exact Correctness \n")

print("\nPredicted Truth vs Ground Truth - ",pred_test)



In [ ]:
pprint(test_truth)
pprint(pred_truth)


In [ ]:
def calculate_deviation(predictions, ground_truth):
    assert len(predictions) == len(ground_truth), "Prediction and ground truth lists must have the same length."
    
    new_array = (ground_truth - predictions)
    ma=np.max(new_array)
    av=np.mean(new_array)
    
    # Compute squared errors
    squared_errors = [(p - gt) ** 2 for p, gt in zip(predictions, ground_truth)]
    
    # Calculate mean of squared errors
    mse = sum(squared_errors) / len(ground_truth)
    
    return ma,av,mse

ma,av,mse=calculate_deviation(pred_truth,test_truth)

print("Deviation Correctness - Prediction \n")

print('\nMax Deviation - ',ma)
print('\nAvg Deviation - ',av)
print('\nMean Square Deviation - ',mse)

In [ ]:
import matplotlib.pyplot as plt
from collections import Counter
import numpy as np

def plot_occurrences(numbers):
    """
    Plots a horizontal bar graph of the occurrences of elements in a list.

    Parameters:
        numbers (list): A list of numbers to analyze.
    """
    # Count occurrences of each element
    occurrences = Counter(numbers)
    

    # Extract keys (unique numbers) and their corresponding values (counts)
    elements = list(occurrences.keys())
    counts = list(occurrences.values())
    print(np.mean(counts))

    # Plot a horizontal bar graph
    plt.figure(figsize=(8, 5))
    plt.barh(elements, counts, color="skyblue")
    plt.xlabel("Occurrences")
    plt.ylabel("Elements")
    plt.title("Occurrences of Elements in the List")
    plt.grid(axis="x", linestyle="--", alpha=0.7)
    plt.show()
    
def plot_frame(pred):
    """
    Plots a horizontal bar graph of the occurrences of elements in a list.

    Parameters:
        numbers (list): A list of numbers to analyze.
    """
    # Count occurrences of each element
    
    ele=[i for i in range(len(pred))]
    # Plot a horizontal bar graph
    plt.figure(figsize=(8, 5))
    plt.barh(ele, pred, color="skyblue")
    plt.xlabel("Occurrences")
    plt.ylabel("Elements")
    plt.title("Occurrences of Elements in the List")
    plt.grid(axis="x", linestyle="--", alpha=0.7)
    plt.show()

plot_frame(predictions[2])
print(predictions)
# print(np.sum(predictions, axis=1))